In [27]:
import numpy as np
from tensorflow.python.keras.layers import GlobalAveragePooling2D, Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import ResNet50V2
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.regularizers import l2
from tensorflow import keras
# from tensorflow.keras import Sequential
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0, 1, 2, 3'

In [16]:
def load_CIFAR10(num_class):
    """
    Load CIFAR10 dataset and data preprocessing.
    :param data_dir: Path where to find the data file.
    :return: Tuple of numpy arrays: (x_train, y_train), (x_test, y_test).
    """
    # Initialize Data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    y_train = tf.keras.utils.to_categorical(y_train, num_class)
    y_test = tf.keras.utils.to_categorical(y_test, num_class)
    
    return (x_train, y_train), (x_test, y_test)

(x, y), (x_test, y_test) = load_CIFAR10(10)

In [29]:
def get_Resnet50(num_classes=10):
    """Define and return the ResNet50V2 model.
    Args:
        input_shape: a tensor defined the data shape.
        num_classes: the number of classes.
    Returns: 
        a keras model object named ResNet50V2.
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=x[0].shape),
        keras.layers.Dense(200, activation='relu', kernel_regularizer=keras.regularizers.l1()), 
        keras.layers.Dense(100, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)), 
        keras.layers.Dense(50, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(0.01)), 
        keras.layers.Dense(10, activation='softmax')])
    
    return model

model = get_Resnet50()

In [2]:
initial_model = get_Resnet50()
for j in range(3):
    deltas = []
    for i in range(4):
        model = get_Resnet50()
        model.set_weights(initial_model.get_weights())
        model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
                      metrics=['accuracy'])
        model.fit(x[i: i+10000],
                  y[i: i+10000],
                  batch_size=32,
                  epochs=30,
                  verbose=1)
        delta = np.array(initial_model.get_weights()) - \
            np.array(model.get_weights())
        deltas.append(delta)
        model.save('./model/keras_cifar10_trained_model_' + str(i) + '.h5')
    delt_av = (deltas[0] * 1.5 + deltas[1] * 0.5 + deltas[2] * 0.5 + deltas[3] * 0.5) / 3
    new_weights = np.array(initial_model.get_weights()) - delt_av
    initial_model.set_weights(new_weights)

NameError: name 'get_Resnet50' is not defined

In [ ]:
initial_model 

In [32]:
def compute_pairwise_distances(x, y):
    """Computes the squared pairwise Euclidean distances between x and y.
    Args:
      x: a tensor of shape [num_x_samples, num_features]
      y: a tensor of shape [num_y_samples, num_features]
    Returns:
      a distance matrix of dimensions [num_x_samples, num_y_samples].
    Raises:
      ValueError: if the inputs do no matched the specified dimensions.
    """

    if not len(x.get_shape()) == len(y.get_shape()) == 2:
        raise ValueError('Both inputs should be matrices.')

    if x.get_shape().as_list()[1] != y.get_shape().as_list()[1]:
        raise ValueError('The number of features should be the same.')

    norm = lambda x: tf.reduce_sum(tf.square(x), 1)

    return tf.transpose(norm(tf.expand_dims(x, 2) - tf.transpose(y)))


def gaussian_kernel_matrix(x, y, sigmas):
    r"""Computes a Guassian Radial Basis Kernel between the samples of x and y.
    We create a sum of multiple gaussian kernels each having a width sigma_i.
    Args:
      x: a tensor of shape [num_samples, num_features]
      y: a tensor of shape [num_samples, num_features]
      sigmas: a tensor of floats which denote the widths of each of the
        gaussians in the kernel.
    Returns:
      A tensor of shape [num_samples{x}, num_samples{y}] with the RBF kernel.
    """
    beta = 1. / (2. * (tf.expand_dims(sigmas, 1)))

    dist = compute_pairwise_distances(x, y)

    s = tf.matmul(beta, tf.reshape(dist, (1, -1)))

    return tf.reshape(tf.reduce_sum(tf.exp(-s), 0), tf.shape(dist))


def maximum_mean_discrepancy(x, y, kernel=gaussian_kernel_matrix):
    '''
    Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.
    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of
    the distributions of x and y. Here we use the kernel two sample estimate
    using the empirical mean of the two distributions.
    MMD^2(P, Q) = || \E{\phi(x)} - \E{\phi(y)} ||^2
                = \E{ K(x, x) } + \E{ K(y, y) } - 2 \E{ K(x, y) },
    where K = <\phi(x), \phi(y)>,
      is the desired kernel function, in this case a radial basis kernel.
    Args:
        x: a tensor of shape [num_samples, num_features]
        y: a tensor of shape [num_samples, num_features]
        kernel: a function which computes the kernel in MMD. Defaults to the
                GaussianKernelMatrix.
    Returns:
        a scalar denoting the squared maximum mean discrepancy loss.
    '''
    with tf.name_scope('MaximumMeanDiscrepancy'):
        # \E{ K(x, x) } + \E{ K(y, y) } - 2 \E{ K(x, y) }
        cost = tf.reduce_mean(kernel(x, x))
        cost += tf.reduce_mean(kernel(y, y))
        cost -= 2 * tf.reduce_mean(kernel(x, y))

        # We do not allow the loss to become negative.
        cost = tf.where(cost > 0, cost, 0, name='value')
    return cost


def mmd_loss(source_samples, target_samples, weight):
    """Adds a similarity loss term, the MMD between two representations.
    This Maximum Mean Discrepancy (MMD) loss is calculated with a number of
    different Gaussian kernels.
    Args:
      source_samples: a tensor of shape [num_samples, num_features].
      target_samples: a tensor of shape [num_samples, num_features].
      weight: the weight of the MMD loss.
      scope: optional name scope for summary tags.
    Returns:
      a scalar tensor representing the MMD loss value.
    """
    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    gaussian_kernel = partial(
        gaussian_kernel_matrix, sigmas=tf.constant(sigmas))

    loss_value = maximum_mean_discrepancy(
        source_samples, target_samples, kernel=gaussian_kernel)
    loss_value = tf.maximum(1e-4, loss_value) * weight

    return loss_value

In [21]:
for i in range(5): 
    i = i % 5   
    x_train, y_train = x[i:i+5000], y[i:i+5000]
    """model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
                loss=tf.keras.losses.categorical_crossentropy,
                metrics = ["accuracy"])"""
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=5e-5),
                  metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model.fit(x_train, y_train,
                epochs=30,
                batch_size=64,
                shuffle=True,
                verbose=1)

Train on 5000 samples
Epoch 1/30
5000/5000 [==============================] - 9s 2ms/sample - loss: 2.4465 - categorical_accuracy: 0.1214 - precision_6: 0.1768 - recall_6: 0.0070
Epoch 2/30
5000/5000 [==============================] - 3s 651us/sample - loss: 2.2185 - categorical_accuracy: 0.1840 - precision_6: 0.4690 - recall_6: 0.0106
Epoch 3/30
5000/5000 [==============================] - 3s 643us/sample - loss: 2.0956 - categorical_accuracy: 0.2558 - precision_6: 0.5774 - recall_6: 0.0306
Epoch 4/30
5000/5000 [==============================] - 3s 651us/sample - loss: 1.9641 - categorical_accuracy: 0.3258 - precision_6: 0.6217 - recall_6: 0.0608
Epoch 5/30
5000/5000 [==============================] - 3s 647us/sample - loss: 1.7767 - categorical_accuracy: 0.4028 - precision_6: 0.7245 - recall_6: 0.1152
Epoch 6/30
5000/5000 [==============================] - 3s 659us/sample - loss: 1.6367 - categorical_accuracy: 0.4508 - precision_6: 0.7438 - recall_6: 0.1690
Epoch 7/30
5000/5000 [====

In [33]:
def sobel(img_set):
    ret = np.empty(img_set.shape)
    for i, img in enumerate(img_set):
        grad_x = cv.Sobel(np.float32(img), cv.CV_32F, 1, 0)
        grad_y = cv.Sobel(np.float32(img), cv.CV_32F, 0, 1)
        gradx = cv.convertScaleAbs(grad_x)
        grady = cv.convertScaleAbs(grad_y)
        gradxy = cv.addWeighted(gradx, 0.5, grady, 0.5, 0)
        ret[i, :] = gradxy
    return ret

In [34]:
def diff_Mem_attack(x_, y_true, m_true, target_model, non_Mem_Generator=sobel):
    '''
    Attck the target with BLINDMI-DIFF-W, BLINDMI-DIFF with gernerated non-member.
    The non-member is generated by randomly chosen data and the number is 20 by default.
    If the data has been shuffled, please directly remove the process of shuffling.
    :param target_model: the model that will be attacked
    :param x_: the data that target model may used for training
    :param y_true: the label of x_
    :param m_true: one of 0 and 1, which represents each of x_ has been trained or not.
    :param non_Mem_Generator: the method to generate the non-member data. The default non-member generator
    is Sobel.
    :return:  Tensor arrays of results
    '''

    y_pred = target_model.predict(x_)
    mix = np.c_[y_pred[y_true.astype(bool)], np.sort(y_pred, axis=1)[:, ::-1][:, :2]]

    nonMem_index = np.random.randint(0, x_.shape[0], size=20)
    nonMem_pred = target_model.predict(non_Mem_Generator(x_[nonMem_index]))
    nonMem = tf.convert_to_tensor(np.c_[nonMem_pred[y_true[nonMem_index].astype(bool)],
                                        np.sort(nonMem_pred, axis=1)[:, ::-1][:, :2]])

    data = tf.data.Dataset.from_tensor_slices((mix, m_true)).shuffle(buffer_size=x_.shape[0]).\
        batch(20).prefetch(tf.data.experimental.AUTOTUNE)

    m_pred, m_true = [], []
    mix_shuffled = []
    for (mix_batch, m_true_batch) in data:
        m_pred_batch = np.ones(mix_batch.shape[0])
        m_pred_epoch = np.ones(mix_batch.shape[0])
        nonMemInMix = True
        while nonMemInMix:
            mix_epoch_new = mix_batch[m_pred_epoch.astype(bool)]
            dis_ori = mmd_loss(nonMem, mix_epoch_new, weight=1)
            nonMemInMix = False
            for index, item in tqdm(enumerate(mix_batch)):
                if m_pred_batch[index] == 1:
                    nonMem_batch_new = tf.concat([nonMem, [mix_batch[index]]], axis=0)
                    mix_batch_new = tf.concat([mix_batch[:index], mix_batch[index+1:]], axis=0)
                    m_pred_without = np.r_[m_pred_batch[:index], m_pred_batch[index+1:]]
                    mix_batch_new = mix_batch_new[m_pred_without.astype(bool, copy=True)]
                    dis_new = mmd_loss(nonMem_batch_new, mix_batch_new, weight=1)
                    if dis_new > dis_ori:
                        nonMemInMix = True
                        m_pred_epoch[index] = 0
            m_pred_batch = m_pred_epoch.copy()

        mix_shuffled.append(mix_batch)
        m_pred.append(m_pred_batch)
        m_true.append(m_true_batch)
    return np.concatenate(m_true, axis=0), np.concatenate(m_pred, axis=0), \
           np.concatenate(mix_shuffled, axis=0), nonMem

In [35]:
def evaluate_attack(m_true, m_pred):
    accuracy = tf.keras.metrics.Accuracy()
    precision = tf.keras.metrics.Precision()
    recall = tf.keras.metrics.Recall()
    accuracy.update_state(m_true, m_pred)
    precision.update_state(m_true, m_pred)
    recall.update_state(m_true, m_pred)
    F1_Score = 2 * (precision.result() * recall.result()) / (precision.result() + recall.result())
    print('accuracy:%.4f precision:%.4f recall:%.4f F1_Score:%.4f'
          % (accuracy.result(), precision.result(), recall.result(), F1_Score))

In [25]:
def load_CIFAR10(num_class):
    """
    Load CIFAR10 dataset and data preprocessing.
    :param data_dir: Path where to find the data file.
    :return: Tuple of numpy arrays: (x_train, y_train), (x_test, y_test).
    """
    # Initialize Data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    y_train = tf.keras.utils.to_categorical(y_train, num_class)
    m_train = np.ones(y_train.shape[0])
    y_test = tf.keras.utils.to_categorical(y_test, num_class)
    m_test = np.zeros(y_test.shape[0])
    member = np.r_[m_train, m_test]
    
    return (x_train, y_train), (x_test, y_test), member

(x, y), (x_test, y_test), num = load_CIFAR10(10)

In [38]:
import cv2 as cv
from functools import partial
from tqdm import tqdm


print(x.shape, x_test.shape)
print(y.shape, y_test.shape)
num1 = np.zeros(10000)
num2 = np.ones(10000)
n = np.vstack((num1[0:10000].reshape((10000, 1)), num2[: 10000].reshape(10000, 1)))
m_true, m_pred, mix, nonMem = diff_Mem_attack(np.r_[x[: 10000], x_test],
                                              np.r_[y[: 10000], y_test],
                                              n, model)
m_pred = m_pred.reshape((20000, 1))
evaluate_attack(m_true, m_pred)

(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 10) (10000, 10)


20it [00:00, 185.31it/s]
20it [00:00, 759.90it/s]
20it [00:00, 252.27it/s]
20it [00:00, 525.24it/s]
20it [00:00, 202.06it/s]
20it [00:00, 513.95it/s]
20it [00:00, 742.29it/s]
20it [00:00, 1057.43it/s]
20it [00:00, 201.41it/s]
20it [00:00, 470.41it/s]
20it [00:00, 668.51it/s]
20it [00:00, 246.94it/s]
20it [00:00, 529.85it/s]
20it [00:00, 662.15it/s]
20it [00:00, 247.12it/s]
20it [00:00, 528.55it/s]
20it [00:00, 667.37it/s]
20it [00:00, 1091.37it/s]
20it [00:00, 248.82it/s]
20it [00:00, 1903.60it/s]
20it [00:00, 247.83it/s]
20it [00:00, 667.93it/s]
20it [00:00, 249.00it/s]
20it [00:00, 351.56it/s]
20it [00:00, 248.74it/s]
20it [00:00, 668.91it/s]
20it [00:00, 237.07it/s]
20it [00:00, 441.35it/s]
20it [00:00, 589.68it/s]
20it [00:00, 248.52it/s]
20it [00:00, 530.75it/s]
20it [00:00, 233.47it/s]
20it [00:00, 590.78it/s]
20it [00:00, 248.35it/s]
20it [00:00, 486.19it/s]
20it [00:00, 237.69it/s]
20it [00:00, 439.25it/s]
20it [00:00, 674.13it/s]
20it [00:00, 245.01it/s]
20it [00:00, 476.47it/

accuracy:0.4999 precision:0.4998 recall:0.3578 F1_Score:0.4170


In [28]:
evaluate_attack(m_true, m_pred)

accuracy:0.5321 precision:0.3299 recall:0.3914 F1_Score:0.3580


In [29]:
print(x.shape, x_test.shape)
print(y.shape, y_test.shape)
print(num.shape, num[:5000].shape, num[50000: 60000].shape)
n = np.vstack((num[5000:10000].reshape((5000, 1)), num[50000: 60000].reshape(10000, 1)))
m_true, m_pred, mix, nonMem = diff_Mem_attack(np.r_[x[5000: 10000], x_test],
                                              np.r_[y[5000: 10000], y_test],
                                              n, model)
m_pred = m_pred.reshape((15000, 1))
evaluate_attack(m_true, m_pred)

(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 10) (10000, 10)
(60000,) (5000,) (10000,)


20it [00:00, 217.77it/s]
20it [00:00, 375.10it/s]
20it [00:00, 581.93it/s]
20it [00:00, 244.62it/s]
20it [00:00, 322.06it/s]
20it [00:00, 411.91it/s]
20it [00:00, 467.82it/s]
20it [00:00, 234.74it/s]
20it [00:00, 248.30it/s]
20it [00:00, 555.07it/s]
20it [00:00, 238.62it/s]
20it [00:00, 303.21it/s]
20it [00:00, 612.76it/s]
20it [00:00, 251.47it/s]
20it [00:00, 392.71it/s]
20it [00:00, 477.34it/s]
20it [00:00, 421.35it/s]
20it [00:00, 241.24it/s]
20it [00:00, 383.09it/s]
20it [00:00, 713.54it/s]
20it [00:00, 240.67it/s]
20it [00:00, 438.41it/s]
20it [00:00, 517.64it/s]
20it [00:00, 575.80it/s]
20it [00:00, 641.84it/s]
20it [00:00, 745.61it/s]
20it [00:00, 222.67it/s]
20it [00:00, 373.35it/s]
20it [00:00, 244.24it/s]
20it [00:00, 358.67it/s]
20it [00:00, 468.03it/s]
20it [00:00, 504.90it/s]
20it [00:00, 562.10it/s]
20it [00:00, 191.45it/s]
20it [00:00, 363.02it/s]
20it [00:00, 385.56it/s]
20it [00:00, 234.68it/s]
20it [00:00, 366.90it/s]
20it [00:00, 422.52it/s]
20it [00:00, 480.18it/s]


accuracy:0.5367 precision:0.3351 recall:0.3960 F1_Score:0.3630


In [33]:
n = np.vstack((num[:20000].reshape((20000, 1)), num[50000: 60000].reshape(10000, 1)))
m_true, m_pred, mix, nonMem = diff_Mem_attack(np.r_[x[:20000], x_test],
                                              np.r_[y[:20000], y_test],
                                              n, model)
m_pred = m_pred.reshape((30000, 1))
evaluate_attack(m_true, m_pred)

20it [00:00, 210.51it/s]
20it [00:00, 330.10it/s]
20it [00:00, 389.92it/s]
20it [00:00, 421.29it/s]
20it [00:00, 236.84it/s]
20it [00:00, 373.87it/s]
20it [00:00, 400.68it/s]
20it [00:00, 243.77it/s]
20it [00:00, 437.98it/s]
20it [00:00, 517.43it/s]
20it [00:00, 640.45it/s]
20it [00:00, 755.53it/s]
20it [00:00, 245.98it/s]
20it [00:00, 400.08it/s]
20it [00:00, 435.86it/s]
20it [00:00, 246.64it/s]
20it [00:00, 322.16it/s]
20it [00:00, 399.96it/s]
20it [00:00, 248.37it/s]
20it [00:00, 372.00it/s]
20it [00:00, 392.71it/s]
20it [00:00, 425.70it/s]
20it [00:00, 236.15it/s]
20it [00:00, 399.85it/s]
20it [00:00, 432.67it/s]
20it [00:00, 243.01it/s]
20it [00:00, 341.35it/s]
20it [00:00, 275.36it/s]
20it [00:00, 242.92it/s]
20it [00:00, 475.08it/s]
20it [00:00, 746.11it/s]
20it [00:00, 242.78it/s]
20it [00:00, 333.78it/s]
20it [00:00, 402.60it/s]
20it [00:00, 227.47it/s]
20it [00:00, 281.20it/s]
20it [00:00, 354.12it/s]
20it [00:00, 352.31it/s]
20it [00:00, 399.17it/s]
20it [00:00, 202.03it/s]


accuracy:0.5682 precision:0.7278 recall:0.5629 F1_Score:0.6348


In [31]:
n = np.vstack((num[5000:10000].reshape((5000, 1)), num[50000: 60000].reshape(10000, 1)))
m_true, m_pred, mix, nonMem = diff_Mem_attack(np.r_[x[15000: 20000], x_test],
                                              np.r_[y[15000: 20000], y_test],
                                              n, model)
m_pred = m_pred.reshape((15000, 1))
evaluate_attack(m_true, m_pred)

20it [00:00, 213.74it/s]
20it [00:00, 443.40it/s]
20it [00:00, 478.20it/s]
20it [00:00, 521.81it/s]
20it [00:00, 589.62it/s]
20it [00:00, 665.67it/s]
20it [00:00, 759.16it/s]
20it [00:00, 245.80it/s]
20it [00:00, 310.88it/s]
20it [00:00, 524.33it/s]
20it [00:00, 338.95it/s]
20it [00:00, 239.36it/s]
20it [00:00, 405.12it/s]
20it [00:00, 248.93it/s]
20it [00:00, 328.10it/s]
20it [00:00, 249.03it/s]
20it [00:00, 407.36it/s]
20it [00:00, 431.53it/s]
20it [00:00, 616.64it/s]
20it [00:00, 621.65it/s]
20it [00:00, 832.05it/s]
20it [00:00, 225.11it/s]
20it [00:00, 394.62it/s]
20it [00:00, 434.83it/s]
20it [00:00, 478.50it/s]
20it [00:00, 552.97it/s]
20it [00:00, 175.54it/s]
20it [00:00, 350.62it/s]
20it [00:00, 462.78it/s]
20it [00:00, 709.98it/s]
20it [00:00, 1055.74it/s]
20it [00:00, 1244.58it/s]
20it [00:00, 242.24it/s]
20it [00:00, 396.59it/s]
20it [00:00, 437.70it/s]
20it [00:00, 244.26it/s]
20it [00:00, 400.74it/s]
20it [00:00, 475.42it/s]
20it [00:00, 525.58it/s]
20it [00:00, 590.81it/s

accuracy:0.5266 precision:0.3308 recall:0.4108 F1_Score:0.3665


In [ ]:
n = np.vstack((num[5000:10000].reshape((5000, 1)), num[50000: 60000].reshape(10000, 1)))
m_true, m_pred, mix, nonMem = diff_Mem_attack(np.r_[x[20000: 25000], x_test],
                                              np.r_[y[20000: 25000], y_test],
                                              n, model)
m_pred = m_pred.reshape((15000, 1))
evaluate_attack(m_true, m_pred)

20it [00:00, 205.94it/s]
20it [00:00, 342.78it/s]
20it [00:00, 348.77it/s]
20it [00:00, 216.46it/s]
20it [00:00, 231.10it/s]
20it [00:00, 365.30it/s]
20it [00:00, 655.35it/s]
20it [00:00, 884.38it/s]
20it [00:00, 1068.31it/s]
20it [00:00, 245.32it/s]
20it [00:00, 300.81it/s]
20it [00:00, 480.63it/s]
20it [00:00, 245.73it/s]
20it [00:00, 408.86it/s]
20it [00:00, 322.52it/s]
20it [00:00, 243.36it/s]
20it [00:00, 465.57it/s]
20it [00:00, 669.23it/s]
20it [00:00, 761.13it/s]
20it [00:00, 245.60it/s]
20it [00:00, 428.87it/s]
20it [00:00, 439.22it/s]
20it [00:00, 246.09it/s]
20it [00:00, 249.03it/s]
20it [00:00, 326.33it/s]
20it [00:00, 207.49it/s]
20it [00:00, 317.17it/s]
20it [00:00, 390.85it/s]
20it [00:00, 185.88it/s]
20it [00:00, 345.36it/s]
20it [00:00, 527.55it/s]
20it [00:00, 248.01it/s]
20it [00:00, 436.91it/s]
20it [00:00, 516.08it/s]
20it [00:00, 565.08it/s]
20it [00:00, 240.18it/s]
20it [00:00, 399.76it/s]
20it [00:00, 238.53it/s]
20it [00:00, 335.34it/s]
20it [00:00, 480.72it/s]

accuracy:0.5041 precision:0.3341 recall:0.4914 F1_Score:0.3978
